In [1]:
class RS:
    def __init__(self,q,n,k, K, g):
        self.q = q
        self.n = n
        self.k = k
        self.x = [0 for i in range(n)]
        self.d = self.n - self.k + 1
        self.t = floor((self.d - 1)/2.0)
        self.K = K
        self.g = g
        self.x = [self.g^i for i in range(n-1)]
        self.x.insert(0,0)

        
    def encode(self,M):
        v = vector(self.K,[sum([(self.x[j]^(self.k-i-1))*M[i] for i in range(self.k)]) for j in range(self.n)])
        return v
        
    def distance_min(self):
        info_space = VectorSpace(self.K,self.k)
        w_min = self.n

        for M in info_space:
            w_tmp = self.encode(self.K,M).hamming_weight()
            if w_tmp > 0 and w_tmp<w_min:
                w_min = w_tmp
                
        return w_min
    
    def decode_BW(self,U):
        M = Matrix(self.K, self.n, self.n + 1) 
        for i in range(self.n):
            for j in range(self.t + 1):
                M[i,j] = U[i]*(self.x[i])^j
            for j in range(self.n - self.t):
                M[i,self.t+1+j] = -(self.x[i])^(j)
        #print(M)
        
        coefs = M.right_kernel().basis_matrix().rows()[0]
        R.<X> = PolynomialRing(self.K)
        A = 0
        B = 0
        tmp = 1
        for i in range(self.t):
            A += coefs[i]*tmp
            tmp *= X
        tmp = 1
        for i in range(self.k + self.t - 1):
            B += coefs[self.t + i] * tmp
            tmp *= X
        r = B//A
        coefs = r.list() # https://ask.sagemath.org/question/26907/get-the-coefficients-from-the-polynomial/
        
        while len(coefs)<self.n:
            coefs.append(0)
        
        return coefs
    

In [2]:
def canal(x, nb_err, K):
    n = len(x)
    I = Subsets(n,nb_err).random_element()
    y = []
    for i in range(n):
        y.append(x[i])
        if i+1 in I:
            err = 0
            while err == 0:
                err = K.random_element()
            y[i] += err
    return vector(K,y)
 

In [3]:
K.<g> = GF(8)
R = RS(8,6,4,K,g)

M = vector(K, [R.K.random_element() for i in range(R.k)])
m = R.encode(M)
print("Message :", m)
U = canal(m, R.t, K)
print("Received : ", U)
m1 = R.decode_BW(U)
print("decoded :", m1)


Message : (0, 1, g, g, 0, g^2 + 1)
Received :  (0, 1, g^2 + g, g, 0, g^2 + 1)
decoded : [g^2 + 1, 0, 0, g^2, 0, 0]
